In [2]:
import numpy as np
from sklearn.preprocessing import StandardScaler
import pickle
import pandas as pd

In [2]:
pred_data = np.load(r'results\stock_000001SZustd_50_1_PatchTST_stock_custom_ftMS_sl50_ll25_pl1_dm128_nh8_el3_dl1_df256_fc1_ebtimeF_dtTrue_Exp_0\pred.npy')
print(pred_data.shape)
pred_data = pred_data.reshape((pred_data.shape[0]))
print(pred_data[:15])

true_data = np.load(r'results\stock_000001SZustd_50_1_PatchTST_stock_custom_ftMS_sl50_ll25_pl1_dm128_nh8_el3_dl1_df256_fc1_ebtimeF_dtTrue_Exp_0\true.npy')
true_data = true_data.reshape((true_data.shape[0]))
print(true_data[:15])

(800, 1, 1)
[-0.24318472 -0.10326846  0.14551774  0.09388182  0.16411474  0.09579294
  0.08957873 -0.23152643 -0.16371813  0.00643662  0.04437798  0.08180664
  0.16832884  0.50132424  0.14367586]
[-0.28807533  0.2936211   0.84149134  0.04105194  0.12143223  0.17425466
 -0.6780516  -0.12085529 -1.0155877  -1.124596    0.01754558  0.44627866
  2.3314888   0.33241615  1.5396876 ]


In [3]:
#load scaler
with open(r'scaler\stock_000001SZustd\stock_000001SZustd_50_1.pkl', 'rb') as f:
    scaler = pickle.load(f)

c:\Users\Vincent\.conda\envs\Pytorch_1_11\lib\site-packages\sklearn\base.py:347: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.2.2 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [4]:
tc =  np.load('scaler\stock_000001SZustd\stock_000001SZustd_50_1.npy')
print(tc.shape)

(2, 10)


In [5]:
print(pred_data.shape)
print(pred_data[:10])

(800,)
[-0.24318472 -0.10326846  0.14551774  0.09388182  0.16411474  0.09579294
  0.08957873 -0.23152643 -0.16371813  0.00643662]


In [6]:
pred_tmp_t = np.full((pred_data.shape[0], 9), 0, np.float32)
print(pred_tmp_t.shape)
pred_tmp_t = np.insert(pred_tmp_t, 9, pred_data, axis=1)
print(pred_tmp_t.shape)


(800, 9)
(800, 10)


In [7]:
#get the orginal 
pred_orig = scaler.inverse_transform(pred_tmp_t)[:,-1]
print(pred_orig[:10])

[-0.5323521  -0.16628733  0.48461524  0.3495195   0.53327084  0.3545196
  0.33826125 -0.50185037 -0.32444263  0.12073547]


In [8]:
def data_inverse(label_c, original_dimension, original_position, scaler): #original dimensino(2D) without the label_c column
    temp_matrix = np.full(original_dimension, 0, np.float32)
    temp_matrix = np.insert(temp_matrix, original_position, label_c, axis=1)
    return scaler.inverse_transform(temp_matrix)[:, original_position]

In [9]:
# print(pred_orig[:10] == data_inverse(pred_data, (pred_data.shape[0], 9), 9, scaler)[:10])

In [10]:
true_y = data_inverse(true_data, (true_data.shape[0], 9), 9, scaler)
print(true_y.shape)

A_x = np.genfromtxt('PatchTST_supervised\stock_data\stock_000001SZustd.csv', delimiter=',', skip_header=1)
A_x = A_x.astype(np.float32)
A_x = A_x[: , 1:]
print(A_x[:5])
print(A_x.shape)
print(A_x.dtype)
print(A_x[:,0].shape)

(800,)
[[ 3.6099999e+00  1.0841413e+05  6.7332305e+04 -2.2471910e+00
   1.4354067e+00 -1.6722408e-01  4.2919999e-01  4.2918077e-01
  -3.1142139e+00 -2.5605137e+00]
 [-1.9000000e+00  1.0686837e+05  6.6813688e+04  3.1198688e+00
   6.2893081e-01  2.5125628e+00 -1.2294000e+00 -1.2293609e+00
   6.9516301e-01  4.1986537e-01]
 [ 0.0000000e+00  7.9396477e+04  4.9387039e+04 -1.5923567e+00
  -9.3750000e-01  3.2679740e-01 -1.7776000e+00 -1.7775663e+00
  -1.3017663e+00 -1.8708645e+00]
 [-2.7500000e+00  1.1148138e+05  6.7292188e+04  1.6181229e-01
  -1.5772871e+00 -3.9087949e+00  1.9450000e-01  1.9450904e-01
  -1.8292873e+00 -1.7309900e-01]
 [-1.9900000e+00  5.4461621e+04  3.2003586e+04 -2.7463651e+00
  -3.5256410e+00 -2.3728814e+00 -1.3827000e+00 -1.3826935e+00
   2.1324158e-01 -1.4597582e+00]]
(4139, 10)
float32
(4139,)


In [18]:
def find_subset_pos(A, B):
    print(A.shape)
    print(B.shape)
    for i in range(A.shape[0] - B.shape[0] + 1):
        if np.array_equal(A[i:i+B.shape[0]], B):
            return i
    return -1

In [19]:
find_subset_pos(A_x[:, 1].astype(np.int16), true_y.astype(np.int16))

(4139,)
(800,)


-1

In [16]:
find_subset_pos(np.array([1,2,3,4]), np.array([4.2]))


-1

In [55]:
#pd date time

s1 = pd.read_csv(r'PatchTST_supervised\stock_data\stock_000001SZustd.csv')
format_string = '%Y%m%d'

In [56]:
ts1 = pd.to_datetime(s1.date, format='%Y%m%d')

In [57]:
ts3 = pd.to_datetime(s1.date, format=format_string)

In [59]:
ts3

0      2005-05-10
1      2005-05-11
2      2005-05-12
3      2005-05-13
4      2005-05-16
          ...    
4134   2022-12-09
4135   2022-12-12
4136   2022-12-13
4137   2022-12-14
4138   2022-12-15
Name: date, Length: 4139, dtype: datetime64[ns]

In [60]:
ts1

0      2005-05-10
1      2005-05-11
2      2005-05-12
3      2005-05-13
4      2005-05-16
          ...    
4134   2022-12-09
4135   2022-12-12
4136   2022-12-13
4137   2022-12-14
4138   2022-12-15
Name: date, Length: 4139, dtype: datetime64[ns]

In [44]:
ts2 = pd.to_datetime(ts1)

In [47]:
ts2.values

array(['2005-05-10T00:00:00.000000000', '2005-05-11T00:00:00.000000000',
       '2005-05-12T00:00:00.000000000', ...,
       '2022-12-13T00:00:00.000000000', '2022-12-14T00:00:00.000000000',
       '2022-12-15T00:00:00.000000000'], dtype='datetime64[ns]')

In [61]:
ts2

0      2005-05-10
1      2005-05-11
2      2005-05-12
3      2005-05-13
4      2005-05-16
          ...    
4134   2022-12-09
4135   2022-12-12
4136   2022-12-13
4137   2022-12-14
4138   2022-12-15
Name: date, Length: 4139, dtype: datetime64[ns]

In [53]:
pd.tseries.frequencies.to_offset('2t')

<2 * Minutes>

In [3]:
df_t = pd.read_csv(r"data/stock_data_us_pctindex/BA_pct_index.csv")

In [4]:
df_t


,date,volume,open_pct_change,high_pct_change,low_pct_change,close_pct_change,index_volume,index_high_pct_change,index_low_pct_change,index_close_pct_change
0,1962-01-03,710775,-0.245734,1.719881,1.500052,2.000023,3590000,-0.667031,-0.466698,0.239569
1,1962-01-04,911250,0.492583,0.241476,-0.492581,-0.980334,4450000,0.195858,0.099460,-0.688877
2,1962-01-05,880875,-0.980344,-2.168627,-4.703017,-1.980219,4630000,-1.089090,-1.561389,-1.387310
3,1962-01-08,473850,-1.980231,-0.738957,1.558458,0.252431,4620000,-1.411632,-1.701515,-0.775195
4,1962-01-09,577125,0.252511,0.992687,0.767345,0.251992,3600000,0.128872,0.968173,0.043401
...,...,...,...,...,...,...,...,...,...,...
15532,2023-09-18,4299200,-0.654623,-0.915026,-0.918411,-1.436743,3161230000,-0.702985,-0.114681,0.072129
15533,2023-09-19,4039900,-1.207258,-1.082199,-1.263536,-0.312012,3614880000,-0.369647,-0.574052,-0.215101
15534,2023-09-20,3738800,0.228822,0.286879,-0.039529,-1.031886,3308450000,0.251237,-0.344834,-0.939479
15535,2023-09-21,4745700,-2.365568,-1.284846,-1.285152,-1.195829,3662340000,-1.912778,-1.640621,-1.640093


In [7]:
df_t2 = df_t[-51:-1]

In [8]:
df_t2

,date,volume,open_pct_change,high_pct_change,low_pct_change,close_pct_change,index_volume,index_high_pct_change,index_low_pct_change,index_close_pct_change
15486,2023-07-13,4291700,-0.628668,-1.200579,-0.285817,-0.119752,3839530000,0.647011,0.585448,0.847015
15487,2023-07-14,4287900,-0.518042,-0.858835,-1.613502,-1.720088,3647450000,0.229777,0.227208,-0.102441
15488,2023-07-17,3621500,-2.073733,-1.285600,-0.502792,-0.586524,3538240000,0.112425,0.118675,0.385538
15489,2023-07-18,5895800,-0.705882,-0.555479,-1.024841,-0.141591,4090010000,0.649695,0.215098,0.711728
15490,2023-07-19,4814100,0.056870,-0.549192,-0.634634,-1.403791,4115670000,0.353558,0.950034,0.235791
15491,2023-07-20,5782200,-1.084688,1.524525,0.268919,2.401723,3761770000,-0.299010,-0.656501,-0.675690
15492,2023-07-21,4053200,2.423022,0.232450,1.360151,-0.847337,3570190000,-0.213380,0.181775,0.032409
15493,2023-07-24,4898300,-0.855581,0.157698,-0.085047,1.921620,3856250000,0.184636,0.121258,0.403415
15494,2023-07-25,6513300,1.009148,-0.518660,-0.307391,-0.810673,3812470000,0.377129,0.245082,0.281467
15495,2023-07-26,24923800,3.716156,8.504789,5.198998,8.724084,3990290000,0.040390,-0.106314,-0.015544


In [10]:
df_t2.to_csv('test_1.csv', index=False)

In [9]:
df_t[-5:]

,date,volume,open_pct_change,high_pct_change,low_pct_change,close_pct_change,index_volume,index_high_pct_change,index_low_pct_change,index_close_pct_change
15532,2023-09-18,4299200,-0.654623,-0.915026,-0.918411,-1.436743,3161230000,-0.702985,-0.114681,0.072129
15533,2023-09-19,4039900,-1.207258,-1.082199,-1.263536,-0.312012,3614880000,-0.369647,-0.574052,-0.215101
15534,2023-09-20,3738800,0.228822,0.286879,-0.039529,-1.031886,3308450000,0.251237,-0.344834,-0.939479
15535,2023-09-21,4745700,-2.365568,-1.284846,-1.285152,-1.195829,3662340000,-1.912778,-1.640621,-1.640093
15536,2023-09-22,4436336,-0.517410,-1.404715,-1.286870,-1.120275,2135953000,-0.418225,-0.292890,-0.229560


In [13]:
result = np.load(r"results\test_PatchTST_stock_custom_pred_ftMS_sl50_ll3_pl1_dm64_nh8_el3_dl1_df128_fc1_ebtimeF_dtTrue_Exp_0\real_prediction.npy")

In [14]:
result

array([[[0.07768882]]], dtype=float32)